## Initial setup

In [2]:
%%capture
!pip install wandb

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import models

import matplotlib.pyplot as plt 
import numpy as np
import random
import time
import os

tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

## Dataset gathering and preparation

In [5]:
# Gather Flowers dataset
train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True
)

AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 256

@tf.function
def scale_resize_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (224, 224)) # Resizing to highest resolution used while training swav
    return (image, label)

training_ds = (
    train_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

testing_ds = (
    validation_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.



Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


## ResNet50 base and a custom classification head

In [6]:
def get_training_model(trainable=False):
    inputs = layers.Input(shape=(224, 224, 3))
    EXTRACTOR = tf.keras.applications.ResNet50(weights="imagenet", include_top=False,
        input_shape=(224, 224, 3))
    EXTRACTOR.trainable = trainable
    x = EXTRACTOR(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(5, activation="softmax")(x)
    classifier = models.Model(inputs=inputs, outputs=x)
    
    return classifier

In [7]:
model = get_training_model()
model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


## Callback

In [8]:
# Early Stopping to prevent overfitting
early_stopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, verbose=2, restore_best_weights=True)

## Without Augmentation

#### Training

In [9]:
# get model and compile
tf.keras.backend.clear_session()
model = get_training_model()
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer="adam")

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train 
history = model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=100,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


Epoch 1/100
13/13 [==============================] - 23s 2s/step - loss: 1.6661 - acc: 0.1978 - val_loss: 1.6145 - val_acc: 0.2600
Epoch 2/100
13/13 [==============================] - 20s 2s/step - loss: 1.5835 - acc: 0.2705 - val_loss: 1.5711 - val_acc: 0.3018
Epoch 3/100
13/13 [==============================] - 20s 2s/step - loss: 1.5418 - acc: 0.3487 - val_loss: 1.5442 - val_acc: 0.2873
Epoch 4/100
13/13 [==============================] - 20s 2s/step - loss: 1.5127 - acc: 0.3561 - val_loss: 1.5242 - val_acc: 0.3327
Epoch 5/100
13/13 [==============================] - 20s 2s/step - loss: 1.4962 - acc: 0.3728 - val_loss: 1.5115 - val_acc: 0.3400
Epoch 6/100
13/13 [==============================] - 21s 2s/step - loss: 1.4815 - acc: 0.3817 - val_loss: 1.5017 - val_acc: 0.3436
Epoch 7/100
13/13 [==============================] - 21s 2s/step - loss: 1.4712 - acc: 0.3859 - val_loss: 1.4934 - val_acc: 0.3455
Epoch 8/100
13/13 [==============================] - 20s 2s/step - loss: 1.4628 - a

#### Evaluation

In [10]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

3/3 [==============================] - 2s 570ms/step - loss: 1.2681 - acc: 0.4836


# Training with Augmentation


#### Augmentation

In [11]:
# Configs
CROP_SIZE = 224
MIN_SCALE = 0.5
MAX_SCALE = 1.

# Experimental options
options = tf.data.Options()
options.experimental_optimization.noop_elimination = True
options.experimental_optimization.map_vectorization.enabled = True
options.experimental_optimization.apply_default_optimizations = True
options.experimental_deterministic = False
options.experimental_threading.max_intra_op_parallelism = 1

In [12]:
@tf.function
def scale_image(image, label):
	image = tf.image.convert_image_dtype(image, tf.float32)
	return (image, label)

@tf.function
def random_apply(func, x, p):
	return tf.cond(
		tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
				tf.cast(p, tf.float32)),
		lambda: func(x),
		lambda: x)
 
@tf.function
def random_resize_crop(image, label):
  # Conditional resizing
  image = tf.image.resize(image, (260, 260))
  # Get the crop size for given min and max scale
  size = tf.random.uniform(shape=(1,), minval=MIN_SCALE*260,
		          maxval=MAX_SCALE*260, dtype=tf.float32)
  size = tf.cast(size, tf.int32)[0]
  # Get the crop from the image
  crop = tf.image.random_crop(image, (size,size,3))
  crop_resize = tf.image.resize(crop, (CROP_SIZE, CROP_SIZE))
  
  return crop_resize, label

@tf.function
def tie_together(image, label):
  # Scale the pixel values
  image, label = scale_image(image , label)
  # random horizontal flip
  image = random_apply(tf.image.random_flip_left_right, image, p=0.5)
  # Random resized crops
  image, label = random_resize_crop(image, label)
  
  return image, label

In [13]:
trainloader = (
	train_ds
	.shuffle(1024)
	.map(tie_together, num_parallel_calls=AUTO)
	.batch(BATCH_SIZE)
	.prefetch(AUTO)
)

trainloader = trainloader.with_options(options)

#### Training

In [14]:
# get model and compile
tf.keras.backend.clear_session()
model = get_training_model()
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer="adam")

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train 
history = model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=100,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


Epoch 1/100
13/13 [==============================] - 21s 2s/step - loss: 1.6408 - acc: 0.2147 - val_loss: 1.6153 - val_acc: 0.2509
Epoch 2/100
13/13 [==============================] - 20s 2s/step - loss: 1.5725 - acc: 0.3135 - val_loss: 1.5623 - val_acc: 0.2836
Epoch 3/100
13/13 [==============================] - 21s 2s/step - loss: 1.5340 - acc: 0.3333 - val_loss: 1.5386 - val_acc: 0.3127
Epoch 4/100
13/13 [==============================] - 20s 2s/step - loss: 1.5101 - acc: 0.3641 - val_loss: 1.5243 - val_acc: 0.3164
Epoch 5/100
13/13 [==============================] - 20s 2s/step - loss: 1.4937 - acc: 0.3724 - val_loss: 1.5114 - val_acc: 0.3291
Epoch 6/100
13/13 [==============================] - 20s 2s/step - loss: 1.4813 - acc: 0.3747 - val_loss: 1.5022 - val_acc: 0.3455
Epoch 7/100
13/13 [==============================] - 20s 2s/step - loss: 1.4717 - acc: 0.3801 - val_loss: 1.4940 - val_acc: 0.3582
Epoch 8/100
13/13 [==============================] - 21s 2s/step - loss: 1.4632 - a

#### Evaluation

In [15]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

3/3 [==============================] - 2s 564ms/step - loss: 1.2689 - acc: 0.4855
